# Problem definition and computing of derivatives
This document is arranged like this :
<ol>
  <li>In the first part we compute the derivatives of logistic loss and regularized logistic loss </li>
  <li>In the second part we solve the logistic regression problem for both cases presented in the first section using ADMM(an alternating direction method of multipliers) approach and we do the program related</li>
  <li>In the third part we solve the logistic regression problem for both cases presented in the first section using BDMM(Basic differential method of multipliers) approach and we do the program related</li>
  <li>We test both method using 2 datasets</li>
</ol>


Let be $S = \{(x_i, y_i)_{i=1}^n \}$ where $x_i \in \mathbb{R}^d, d$ is an integer and $y_i \in \{-1,1\}$ $ \forall i \in \{1,2,...n\}$
<br>
<br>
We define the following functions :
$$\sigma(x) = \frac{1}{1+e^{-x}}, x \in \mathbb{R}$$
<br>
$$h_w(x) = x^{T}\bar{w}+w_0$$ with $ x, \bar{w} \in \mathbb{R}^d, w_o \in \mathbb{R}$ Par la suite nous posons $w = (w_0, \bar{w}) \in \mathbb{R}^{d+1}$
<br>
<br>
We define the logistic loss without regularization as follow :
$$\hat{L_1}(w) = \frac{1}{n}\sum_{i=1}^n{\log(1+e^{-y_i h_w(x_i)}}) \ (1.1)$$
For the case of LASSO the loss is defined as follow :
$$\hat{L_2}(w) = L_1(w) + \lambda \lVert w \rVert_1 \ (1.2)$$
For the case of RIDGE the loss is defined as follow :
$$\hat{L_3}(w) = L_1(w) + \lambda \lVert w \rVert_{2}^{2} \ (1.3) $$
<br>

The functions $L_1$ and $L_3$ are differentiable and their derivative is given by :
$$
\nabla L(\left.w_{0}, w_{1}, \ldots, w_{d}\right)=\left[\begin{array}{c}
\dfrac{\partial L}{\partial w_0}(\left.w_{0}, w_{1}, \ldots, w_{d}\right)\\
\dfrac{\partial L}{\partial w_1}(\left.w_{0}, w_{1}, \ldots, w_{d}\right) \\
\vdots \\
\dfrac{\partial L}{\partial w_d}(\left.w_{0}, w_{1}, \ldots, w_{d}\right) 
\end{array}\right] \ (1.4)
$$

<br>
<br>

Let $ \ l_i(w) = log(1+e^{-y_i h_w(x_i)})$, with the equation (1.1) that come down  to : $L_1(w) = \frac{1}{n} \sum_{i=1}^n{l_i(w)}$
<br>
We know that the derivative of $p(x) = log(1+e^{-u})$ with respect of $x$ and $u = u(x)$ is equal to : 
$p'(x) = \frac{-u'(x)e^{-u(x)}} {1+e^{-u(x)}} = -u'(x)(1-\sigma(u(x)) \ (1.5)$
<br>
Using the relation (1.5) with $u(x) = -y h_w(x)$ we obtain :

$$ \dfrac{\partial l_i}{\partial w_j} =  \dfrac{\partial h_w(x_i)}{\partial w_j} \ast \frac{-y_i e^{-y_i h_w(x_i)}} {1+e^{-y_i h_w(x_i)}} = -y_i\dfrac{\partial h_w(x_i)}{\partial w_j}(1-\sigma(y_i h_w(x_i)) (1.6)$$

<br>

$$ \dfrac{\partial h_w(x_i)}{\partial w_j} = \left\{
    \begin{array}{ll}   
        1 & \mbox{if} j = 0 \\
        x_{ij} & \mbox{else}
    \end{array}
\right. \ (1.7)
\\
$$
<br>
$x_{ij}$ is the j-th component of the vector $x_i$
<br>
So with the equations (1.6) and (1.7) we have :
$$
\dfrac{\partial l_i}{\partial w_j} = \left\{
    \begin{array}{ll}   
        -y_i(1-\sigma(y_i h_w(x_i)) & \mbox{if } j = 0 \\
       -y_i(1-\sigma(y_i h_w(x_i)) \ast x_{ij} & \mbox{else}
    \end{array}
\right.
\\
$$

<br>
<br>

We have : $$\dfrac{\partial L_1}{\partial w_j} = \frac{1}{n}\sum_{i=1}^n{\dfrac{\partial l_i}{\partial w_j}} \left\{
    \begin{array}{ll}   
        \frac{-\sum_{i=1}^n{y_i(1-\sigma(y_i h_w(x_i))}}{n} & \mbox{if } j = 0 \\
        \frac{-\sum_{i=1}^n{y_i(1-\sigma(y_i h_w(x_i))x_{ij}}}{n} & \mbox{else}
    \end{array} \ (1.8)
\right.
\\ 
 $$

For the case of RIDGE we have :
$$\dfrac{\partial L_3}{\partial w_j} = \dfrac{\partial L_1}{\partial w_j} + \lambda \dfrac{\partial \lVert w \rVert_{2}^{2}}{\partial w_j} = \left\{
    \begin{array}{ll}   
        \frac{-\sum_{i=1}^n{y_i(1-\sigma(y_i h_w(x_i))}}{n} + 2 \lambda w_0 & \mbox{if } j = 0 \\
        \frac{-\sum_{i=1}^n{y_i(1-\sigma(y_i h_w(x_i))x_{ij}}}{n}+2\lambda w_j & \mbox{else}
    \end{array}
\right. \ (1.9)
\\ 
 $$


# ADMM (Alternating Direction Method for Multipliers)
Reference :  [Admm-Distr-Stats)](https://fr.scribd.com/document/298976199/Admm-Distr-Stats)

## Solving logistic regression problem without regularization
In this second part we present the ADMM (Alternating Direction Method for Multipliers) approach to solve logistic regression for both cases that we present in the first part.
We consider the loss function $L_1$ in equation (1.1) , the logistic regression problem can be express like that : 
$$ min_{w \in \mathbb{R}^{d+1}} \frac{\sum_{i=1}^n{\log(1+e^{-y_i h_w(x_i)}})}{n}  \ (2.1)$$
This is an optimization problem without constraint, in ADMM form the problem (2.1) can be written as follow :
$$min L_1(w)$$ $$ subject \ to \ w - z = 0 \ (2.2)$$
The augmented lagrangian is defined by :  $$L_\rho(w, z, h) = L_1(w) + h^T (w-z) + \frac{\rho}{2}\lVert w - z \rVert_{2}^{2}$$ 
Using the ADMM algorithm we have the following iterations:
$$w^{k+1} = argmin_{w} \ L_\rho(w, z^{k}, y^{k}) \ (2.3)$$
$$z^{k+1} = argmin_{z} \ L_\rho(w^{k+1}, z, y^{k}) \ (2.4)$$
$$y^{k+1} = y^{k} + \rho (w^{k+1} - z^{k+1}) \ (2.5)$$     
Using the scaled form of ADDM algorithm with $u = \frac{h}{\rho}$ the equations (2.3), (2.4), (2.5) remain to :
$$ w^{k+1} = argmin_{w} \ L_1(w) + \frac{\rho}{2} \lVert w - z^{k} + u^{k} \rVert_{2}^{2} \ (2.6)$$
$$ z^{k+1} = argmin_{z} \lVert w^{k+1} - z + u^{k} \rVert_{2}^{2} \ (2.7)$$
$$u^{k+1} = u^{k} + w^{k+1} - z^{k+1} \ (2.8)$$     
In the first equation (2.6) the objective function is convex so we can use any kind of optimization method to solve it
<br>
In the second equation (2.7) there are not constraints with the variables so the minimum of this function is O and this is obtained when
$$z = w^{k+1} + u^{k} \ (2.9)$$
Proof : To have the scaled form of the ADMM and obtain equations (2.5), (2.6), (2.7) we have the following lines :
Let $ r = w-z$ and $u = \frac{h}{\rho}$ , we have :
$$ h^T (w-z) + \frac{\rho}{2}\lVert w - z \rVert_{2}^{2} = h^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2}$$
$$\Longrightarrow h^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2} = \rho u^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2}$$
$$ \Longrightarrow h^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2} = \frac{\rho}{2}(2u^T r + \lVert r\rVert_{2}^{2}$$
$$ \Longrightarrow h^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2} = \frac{\rho}{2}(\lVert u + r\rVert_{2}^{2} - \lVert u \rVert_{2}^{2})$$
$$ \Longrightarrow h^T r + \frac{\rho}{2}\lVert r\rVert_{2}^{2} = \frac{\rho}{2}(\lVert u + w-z\rVert_{2}^{2} - \lVert u \rVert_{2}^{2}) \ (2.10)$$ 
With equation 2.10 the augmented lagrangian is now given by :
$$L_\rho(w, z, u) = L_1(w) +\frac{\rho}{2}(\lVert u + w-z\rVert_{2}^{2} - \lVert u \rVert_{2}^{2})$$

## Solving logistic regression + RIDGE

We consider the loss function $L_3$ in equation (1.3) , the logistic regression +ridge problem can be express like that : 
$$ min_{w \in \mathbb{R}^{d+1}} L_3(w) \ (3.1)$$

This is an optimization with constraint, in the ADMM form the proble 2.1 can be written as :
$$min \ L_1(w) + \lambda \lVert z \rVert_{2}^{2} \\ subject \ to \ w - z = 0 \ (3.2)$$
the augmented lagrangian is defined by :  $$L_\rho(w, z, h) = L_1(w)+ \lambda \lVert z \rVert_{2}^{2} + h^T (w-z) + \frac{\rho}{2}\lVert w - z \rVert_{2}^{2}$$ 
  
Using the scaled form of ADDM that remains to :
$$ w^{k+1} = argmin_{w} \ L_1(w) + \rho \lVert w - z^{k} + u^{k} \rVert_{2}^{2} \ (3.3)$$
$$ z^{k+1} = argmin_{z} \ \lambda \lVert z \rVert_{2}^{2}+ \frac{\rho}{2} \lVert w^{k+1} - z + u^{k} \rVert_{2}^{2} \ (3.4)$$
$$u^{k+1} = u^{k} + w^{k+1} - z^{k+1} \ (3.5)$$     
In the first equation (3.3) the objective function is convex so we can use any kind of optimization method to solve it
<br>
In the second equation (3.5) this is a least square problem with z as variable we can use a solver for this problem, we have a quadratic function an by solving the gradient of this function equal to the null vector we have a solution defined by :
$$f(z) = \lambda \lVert z \rVert_{2}^{2}+ \frac{\rho}{2} \lVert z - w^{k+1} - u^{k} \rVert_{2}^{2} $$
$$\nabla f(z) = 2\lambda z + \rho (z-w^{k+1}-u^{k})$$
$$\nabla f(z) = 0 \Longrightarrow z =  \frac{\rho (w^{k+1} + u^{k})}{2\lambda + \rho} (3.6) $$

## Solving logistic regression + LASSO with ADMM
In this second part we present the ADMM (Alternating Direction Method for Multipliers) approach to solve logistic regression +LASSO.
We consider the loss function $L_2$ in equation (1.2) , the logistic regression + lasso problem can be express like that : 
$$ min_{w \in \mathbb{R}^{d+1}} L_2(w)  \ (4.1)$$
This is an optimization with constraint, using ADMM form we can written the equation 4.1 as:
$$min \ L_1(w) + \lambda \lVert z \rVert_{1}$$ $$ subject \ to \ w - z = 0 \ (4.2)$$
the augmented lagrangian is defined by :  $$L_\rho(w, z, h) = L_1(w)+ \lambda \lVert z \rVert_{1} + h^T (w-z) + \frac{\rho}{2}\lVert w - z \rVert_{2}^{2}$$  
Using the scaled form of ADDM that remains to :
$$ w^{k+1} = argmin_{w} \ L_1(w) + \rho \lVert w - z^{k} + u^{k} \rVert_{2}^{2} \ (4.3)$$
$$ z^{k+1} = argmin_{z} \ \lambda \lVert z \rVert_{1}+ \frac{\rho}{2} \lVert w^{k+1} - z + u^{k} \rVert_{2}^{2} \ (4.4)$$
$$u^{k+1} = u^{k} + w^{k+1} - z^{k+1} \ (4.5)$$     
In the first equation (4.3) the objective function is convex so we can use any kind of optimization method to solve it
<br>
In the second equation (4.5) the function is not diffentiable so we can solve this problem using a soft thresholding defined as follow :
$$f(z) = \lambda \lVert z \rVert_{1}+ \frac{\rho}{2} \lVert w^{k+1} - z + u^{k} \rVert_{2} $$
$$ z_i = argmin_{z_i} \ f(z_i) \ \forall i \in \{0, 1,..., d\}$$
So this give us  
$$ \forall i \in \{0, 1,..., d\}, \ z_i = S_{\frac{\lambda}{\rho}}(w_{i}^{k+1} + u_{i}^{k}) \ (4.6)$$
Where $$ S_a(v) = \left\{
    \begin{array}{ll}   
        v-a & \mbox{if } v > a \\
        0 & \mbox{if } \lvert v \rvert \le a \\
        a-v & \mbox{else}
    \end{array}
\right.
$$

## Stoping criterion
Let $r^{k}  = w^{k} - z^k$ and $s^k = -\rho (z^k - z^{k-1})$ 
<br>
The stopping criterion of ADMM algorithm is : 
$$ \lVert r^k \rVert_{2}^{2} \le \epsilon^{prim} \ and  \ \lVert s^k \rVert_{2}^{2} \le \epsilon^{dual}$$
With :
$$ \epsilon^{prim}= \sqrt{d+1} \epsilon^{abs} + \epsilon^{rel}max(\lVert w^k \rVert_{2}^{2}, \lVert z^k \rVert_{2}^{2}) \ (5.1)$$
<br>
$$ \epsilon^{dual}= \sqrt{d+1} \epsilon^{abs} + \epsilon^{rel} \lVert u^k \rVert_{2}^{2} \ (5.2)$$

In the equations 5.1 and 5.2 $\epsilon^{abs} \ and \ \epsilon^{rel}$ are fixed by the user 

## Implementation

### Help functions

In [4]:
h_w <- function(w, x){
    p <-length(w)
    return(w[1] + x%*%w[2:p])
}

sigmoid <- function(x){
    return(1/(1+exp(-x)))
}

## the function l1 compute the objective function of equation 2.6
l1 <- function(w, X, y, n, d, u, z, rho){
    f <- 0.0
    for(i in 1:n)
    {
        f <- f + log(1 + exp(-y[i]*h_w(w,X[i,]))) 
    }
    return(f + 0.5*rho*norm(w-z+u, type = "2")^2)
}
## The function grad_l1 compute the gradient of the objective function of equation 2.6
grad_l1 <- function(w, X, y, n, d, u, z, rho){
    grad <- numeric(d+1)
    for(j in 1:d+1)
    {
        s <- 0
        for(i in 1:n)
        {
           
            if(j==1)
            {
                s <- s-y[i]*(1-sigmoid(y[i]*h_w(w, X[i,]))) + rho*(w[j]-z[j]+u[j])
            }
            else
            {
                s <- s-y[i]*(1-sigmoid(y[i]*h_w(w, X[i,])))*X[i,j-1] + rho*(w[j]-z[j]+u[j])
            }

        }
        grad[j] <- s
    }
    return(grad)
}



In [5]:
## ADMM for logistic regression without constraints
addm_logistic_regression <- function(X, y, rho, eps_abs, eps_rel, maxEp){
    ##MaxExp is the total number of iterations that we choose
    n  <- nrow(X)
    d <- ncol(X)
    w <- numeric(d+1)
    z <- numeric(d+1)
    u <- numeric(d+1)
    k <- 1
    while (k<=maxEp)
    {
        optimised <- optim(par = w, X, y, n, d, u, z, rho, fn = l1, gr = grad_l1, method="L-BFGS-B" )
        ## optimised is the function that solves equation 2.6 using newton method w_new at this iteration is w_(k+1)
        ##optimised$par is the vector resulting of this newton optimisation method
        w_new <- optimised$par 
        z_new <- w_new + u #equation 2.9
        u_new <- u + w_new - z_new #equation 2.10
        r <- w_new-z_new
        s <- -rho*(z_new-z)
        eps_prim <- sqrt(d+1)*eps_abs + eps_rel*max(norm(z_new, type = "2"), norm(w_new, type ="2")) # equation 5.1
        eps_dual <- sqrt(d+1)*eps_abs + eps_rel*norm(u_new, type = "2") # equation 5.2
        z <- z_new
        w <- w_new
        u <- u_new
        if((norm(r, type ="2")<=eps_prim) & (norm(s, type = "2")<=eps_dual)){
            # We verify if the stopping criteria is satisfied
            break
        }
        k <- k+1
        print(k)
    }
    return(w)  
}

In [10]:
## ADMM for logistic regression + ridge
addm_logistic_regression_ridge <- function(X, y, rho, lambda, eps_abs, eps_rel, maxEp){
    ##lambda is the penalized coefficient
    n  <- nrow(X)
    d <- ncol(X)
    w <- numeric(d+1)
    z <- numeric(d+1)
    u <- numeric(d+1)
    k <- 1
    while (k<=maxEp)
    {
        optimised <- optim(par = w, X, y, n, d, u, z, rho, fn = l1, gr = grad_l1, method="L-BFGS-B" )
        w_new <- optimised$par
        z_new <- rho(w_new + u)/(2*lambda + rho) #equation 3.6
        u_new <- u + w_new - z_new
        r <- w_new - z_new
        s <- -rho*(z_new - z)
        eps_prim <- sqrt(d+1)*eps_abs + eps_rel*max(norm(z_new, type = "2"), norm(w_new, type ="2"))
        eps_dual <- sqrt(d+1)*eps_abs + eps_rel*norm(u_new, type = "2")
        z <- z_new
        w <- w_new
        u <- u_new
        if((norm(r, type ="2")<=eps_prim) & (norm(s, type = "2")<=eps_dual)){
            # We verify if the stopping criteria is satisfied
            break
        }
        k <- k+1
    }
    return(w)  
}

In [11]:
## This function allows us to compute elements of equation 4.6
soft_threshold <- function(a, v){
    result <- 0
    if(v>a){
        result <- v-a
    }
    if(abs(v)<=a){
        result <- 0
    }
    else{
        result <- a-v
    }
    return(result)
}

In [12]:
addm_logistic_regression_lasso <- function(X, y, rho, lambda, eps_abs, eps_rel, maxEp){
    ##lambda is the penalized coefficient
    n  <- nrow(X)
    d <- ncol(X)
    w <- numeric(d+1)
    z <- numeric(d+1)
    u <- numeric(d+1)
    k <- 1
    while (k<=maxEp)
    {
        optimised <- optim(par = w, X, y, n, d, u, z, rho, fn = l1, gr = grad_l1, method="L-BFGS-B" )
        w_new <- optimised$par
        z_new <- numeric(d+1)
        for(i in 1:d+1){
            z_new[i] <- soft_threshold(lambda/rho, w_new[i] + u_new[i]) #equation 4.6 to compute z_update
        }
        u_new <- u + w_new - z_new
        r <- w_new - z_new
        s <- -rho*(z_new - z)
        eps_prim <- sqrt(d+1)*eps_abs + eps_rel*max(norm(z_new, type = "2"), norm(w_new, type ="2"))
        eps_dual <- sqrt(d+1)*eps_abs + eps_rel*norm(u_new, type = "2")
        z <- z_new
        w <- w_new
        u <- u_new
        if((norm(r, type ="2")<=eps_prim) & (norm(s, type = "2")<=eps_dual)){
            # We verify if the stopping criteria is satisfied
            break
        }
        k <- k+1
    }
    return(w)  
}

In [9]:
sonar <- read.csv('sonar.txt', header =FALSE)
levels(sonar$V61) <- c(-1,1)
set.seed(42)
rows <- sample(nrow(sonar))
sonar <- sonar[rows,]
X <- as.matrix(sonar[, 1:60])
y <- as.vector(sonar[, 61])
y <- as.numeric(y) 

In [23]:
w_test <- addm_logistic_regression(X = X, y = y, rho=1, eps_abs=1e-4,eps_rel=1e-2, maxEp = 100)

[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101


0.8991088
